# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\\$645 - 675) underperformed compared to schools with smaller budgets (\\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

**Note:**
Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import sqlite3

# loading data from python.db database
con = sqlite3.connect('data/python.db')

# Read 'school' and 'student' tables into pandas dataframe
school_data = pd.read_sql_query("SELECT * FROM school", con)
student_data = pd.read_sql_query("SELECT * FROM student", con)


In [7]:
# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,id_x,Student ID,student_name,gender,grade,school_name,reading_score,math_score,id_y,School ID,type,size,budget
0,0,0,Paul Bradley,M,9th,Huang High School,66,79,0,0,District,2917,1910635
1,1,1,Victor Smith,M,12th,Huang High School,94,61,0,0,District,2917,1910635
2,2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,0,District,2917,1910635
3,3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0,District,2917,1910635
4,4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [167]:
# Total number of schools
school_count = school_data['School ID'].count()



In [169]:
# Total number of students
dist_student_count = student_data['Student ID'].count()



In [171]:
# Total budget
total_budget = school_data['budget'].sum()



In [173]:
# Average reading score
avg_reading_score = student_data['reading_score'].mean()



In [175]:
# Average math score
avg_math_score = student_data['math_score'].mean()




In [177]:
# Percentage of passing math (70 or greater)
num_passing_math = student_data[student_data['math_score'] >= 70]['Student ID'].count()



In [179]:
#omg i think this one works now..here i will calc the # of students with a passing math score
perc_pass_math = (num_passing_math / dist_student_count) * 100

In [181]:
# Calculate the number of students with a passing reading score (70 or greater)
num_passing_reading = student_data[student_data['reading_score'] >= 70]['Student ID'].count()

In [183]:
# Calculate the percentage of students with a passing reading score
perc_pass_reading = (num_passing_reading / dist_student_count) * 100

In [185]:
# Calculate the overall passing rate (average of the percentage passing math and reading)
overall_pass = (perc_pass_math + perc_pass_reading) / 2

In [187]:
# Create a District Summary
district_summary = pd.DataFrame({
    "Total Schools": [school_count],
    "Total Students": [dist_student_count],
    "Total Budget": [total_budget],
    "Average Reading Score": [avg_reading_score],
    "Average Math Score": [avg_math_score],
    "% Passing Reading": [perc_pass_reading],
    "% Passing Math": [perc_pass_math],
    "Overall Passing Rate": [overall_pass]
})

# Display the dataframe
print(district_summary)



   Total Schools  Total Students  Total Budget  Average Reading Score  \
0             15           39170      24649428               81.87784   

   Average Math Score  % Passing Reading  % Passing Math  Overall Passing Rate  
0           78.985371          85.805463       74.980853             80.393158  


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

### Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [279]:
#  Sort and display the top five schools in overall passing rate)
# 6. Calculate overall passing rate per school
per_school_summary['Overall Passing Percentage'] = (per_school_summary['% Passing Math'] + per_school_summary['% Passing Reading']) / 2

   


In [283]:
# Calculate total school budget
per_school_summary['Total School Budget'] = school_data_complete.groupby('school_name')['budget'].first()



In [285]:
# Calculate per student budget
per_school_summary['Per Student Budget'] = per_school_summary['Total School Budget'] / per_school_summary['Total Students']




In [287]:
# Cacluate the avg math and reading score
per_school_summary['Average Math Score'] = school_data_complete.groupby('school_name')['math_score'].mean()

per_school_summary['Average Reading Score'] = school_data_complete.groupby('school_name')['reading_score'].mean()




#### Find the passing rate for math and reading (above 70 points)

In [289]:
# Find the total counts of math result
total_math_count = school_data_complete.groupby('school_name')['math_score'].count()
# Find the counts for math result in each school that pass 70 or higher
passing_math_count = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['math_score'].count()
# Calculate the math passing rate
per_school_summary['% Passing Math'] = (passing_math_count / total_math_count) * 100

In [299]:
# Find the total counts of read result
total_reading_count = school_data_complete.groupby('school_name')['reading_score'].count()
# Find the counts for read result in each school that pass 70 or higher
passing_reading_count = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['reading_score'].count()
# Calculate the read passing rate
per_school_summary['% Passing Reading'] = (passing_reading_count / total_reading_count) * 100

### Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [277]:
#  Sort and display the worst five schools in overall passing rate
bottom_schools = per_school_summary_df.sort_values(by='Overall Passing Percentage', ascending=True)

# Display the bottom five schools
print(bottom_schools.head(5))


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [104]:
# Calculate the average math score for 9th grade in each school
grade_9th_data = student_data[student_data['grade'] == '9th']

avg_math_score_9th = grade_9th_data.groupby('school_name')['math_score'].mean()

print(avg_math_score_9th)



school_name
Bailey High School       77.083676
Cabrera High School      83.094697
Figueroa High School     76.403037
Ford High School         77.361345
Griffin High School      82.044010
Hernandez High School    77.438495
Holden High School       83.787402
Huang High School        77.027251
Johnson High School      77.187857
Pena High School         83.625455
Rodriguez High School    76.859966
Shelton High School      83.420755
Thomas High School       83.590022
Wilson High School       83.085578
Wright High School       83.264706
Name: math_score, dtype: float64


In [106]:
# Calculate the average math score for 10th grade in each school
grade_10th_data = student_data[student_data['grade'] == '10th']

avg_math_score_10th = grade_10th_data.groupby('school_name')['math_score'].mean()

print(avg_math_score_10th)



school_name
Bailey High School       76.996772
Cabrera High School      83.154506
Figueroa High School     76.539974
Ford High School         77.672316
Griffin High School      84.229064
Hernandez High School    77.337408
Holden High School       83.429825
Huang High School        75.908735
Johnson High School      76.691117
Pena High School         83.372000
Rodriguez High School    76.612500
Shelton High School      82.917411
Thomas High School       83.087886
Wilson High School       83.724422
Wright High School       84.010288
Name: math_score, dtype: float64


In [114]:
# Calculate the average math score for 11th grade in each school
grade_11th_data = student_data[student_data['grade'] == '11th']

avg_math_score_11th = grade_11th_data.groupby('school_name')['math_score'].mean()

print(avg_math_score_11th)



school_name
Bailey High School       77.515588
Cabrera High School      82.765560
Figueroa High School     76.884344
Ford High School         76.918058
Griffin High School      83.842105
Hernandez High School    77.136029
Holden High School       85.000000
Huang High School        76.446602
Johnson High School      77.491653
Pena High School         84.328125
Rodriguez High School    76.395626
Shelton High School      83.383495
Thomas High School       83.498795
Wilson High School       83.195326
Wright High School       83.836782
Name: math_score, dtype: float64


In [118]:
# Calculate the average math score for 12th grade in each school
grade_12th_data = student_data[student_data['grade'] == '12th']

avg_math_score_12th = grade_12th_data.groupby('school_name')['math_score'].mean()

print(avg_math_score_12th)



school_name
Bailey High School       76.492218
Cabrera High School      83.277487
Figueroa High School     77.151369
Ford High School         76.179963
Griffin High School      83.356164
Hernandez High School    77.186567
Holden High School       82.855422
Huang High School        77.225641
Johnson High School      76.863248
Pena High School         84.121547
Rodriguez High School    77.690748
Shelton High School      83.778976
Thomas High School       83.497041
Wilson High School       83.035794
Wright High School       83.644986
Name: math_score, dtype: float64


In [125]:
avg_math_score_9th = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')['math_score'].mean()
avg_math_score_10th = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')['math_score'].mean()
avg_math_score_11th = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')['math_score'].mean()
avg_math_score_12th = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')['math_score'].mean()

#here I am creating my combied data frame for my series
math_scores_by_grade = pd.DataFrame({
    '9th Grade': avg_math_score_9th,
    '10th Grade': avg_math_score_10th,
    '11th Grade': avg_math_score_11th,
    '12th Grade': avg_math_score_12th
})

# orginal code displayed %s with 6 decimal places
# My line of code below will round to the nearest whole # for a neater look
math_scores_by_grade = math_scores_by_grade.round(0)

#can we go over how to add boarders to tables in class? might be out of our scope


# display my table (hopefully :((
print(math_scores_by_grade)

#yay


                       9th Grade  10th Grade  11th Grade  12th Grade
school_name                                                         
Bailey High School          77.0        77.0        78.0        76.0
Cabrera High School         83.0        83.0        83.0        83.0
Figueroa High School        76.0        77.0        77.0        77.0
Ford High School            77.0        78.0        77.0        76.0
Griffin High School         82.0        84.0        84.0        83.0
Hernandez High School       77.0        77.0        77.0        77.0
Holden High School          84.0        83.0        85.0        83.0
Huang High School           77.0        76.0        76.0        77.0
Johnson High School         77.0        77.0        77.0        77.0
Pena High School            84.0        83.0        84.0        84.0
Rodriguez High School       77.0        77.0        76.0        78.0
Shelton High School         83.0        83.0        83.0        84.0
Thomas High School          84.0  

### Reading Score by Grade 

* Perform the same operations as above for reading scores

In [191]:
# Calculate the average reading score for 9th grade in each school
grade_9th_data = student_data[student_data['grade'] == '9th']

avg_reading_score_9th = grade_9th_data.groupby('school_name')['reading_score'].mean()

print(avg_reading_score_9th)




school_name
Bailey High School       81.303155
Cabrera High School      83.676136
Figueroa High School     81.198598
Ford High School         80.632653
Griffin High School      83.369193
Hernandez High School    80.866860
Holden High School       83.677165
Huang High School        81.290284
Johnson High School      81.260714
Pena High School         83.807273
Rodriguez High School    80.993127
Shelton High School      84.122642
Thomas High School       83.728850
Wilson High School       83.939778
Wright High School       83.833333
Name: reading_score, dtype: float64


In [193]:
# Calculate the average reading score for 10th grade in each school
grade_10th_data = student_data[student_data['grade'] == '10th']

avg_reading_score_10th = grade_10th_data.groupby('school_name')['reading_score'].mean()

print(avg_reading_score_10th)



school_name
Bailey High School       80.907183
Cabrera High School      84.253219
Figueroa High School     81.408912
Ford High School         81.262712
Griffin High School      83.706897
Hernandez High School    80.660147
Holden High School       83.324561
Huang High School        81.512386
Johnson High School      80.773431
Pena High School         83.612000
Rodriguez High School    80.629808
Shelton High School      83.441964
Thomas High School       84.254157
Wilson High School       84.021452
Wright High School       83.812757
Name: reading_score, dtype: float64


In [195]:
# Calculate the average reading score for 11th grade in each school
grade_11th_data = student_data[student_data['grade'] == '11th']

avg_reading_score_11th = grade_11th_data.groupby('school_name')['reading_score'].mean()

print(avg_reading_score_11th)



school_name
Bailey High School       80.945643
Cabrera High School      83.788382
Figueroa High School     80.640339
Ford High School         80.403642
Griffin High School      84.288089
Hernandez High School    81.396140
Holden High School       83.815534
Huang High School        81.417476
Johnson High School      80.616027
Pena High School         84.335938
Rodriguez High School    80.864811
Shelton High School      84.373786
Thomas High School       83.585542
Wilson High School       83.764608
Wright High School       84.156322
Name: reading_score, dtype: float64


In [197]:
# Calculate the average reading score for 12th grade in each school
grade_12th_data = student_data[student_data['grade'] == '12th']

avg_reading_score_12th = grade_12th_data.groupby('school_name')['reading_score'].mean()

print(avg_reading_score_12th)



school_name
Bailey High School       80.912451
Cabrera High School      84.287958
Figueroa High School     81.384863
Ford High School         80.662338
Griffin High School      84.013699
Hernandez High School    80.857143
Holden High School       84.698795
Huang High School        80.305983
Johnson High School      81.227564
Pena High School         84.591160
Rodriguez High School    80.376426
Shelton High School      82.781671
Thomas High School       83.831361
Wilson High School       84.317673
Wright High School       84.073171
Name: reading_score, dtype: float64


In [201]:
avg_reading_score_9th = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')['reading_score'].mean()
avg_reading_score_10th = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')['reading_score'].mean()
avg_reading_11th = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')['reading_score'].mean()
avg_reading_score_12th = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')['reading_score'].mean()


reading_scores_by_grade = pd.DataFrame({
    '9th Grade': avg_reading_score_9th,
    '10th Grade': avg_reading_score_10th,
    '11th Grade': avg_reading_score_11th,
    '12th Grade': avg_reading_score_12th
})

reading_scores_by_grade = reading_scores_by_grade.round(0)

print(reading_scores_by_grade)




                       9th Grade  10th Grade  11th Grade  12th Grade
school_name                                                         
Bailey High School          81.0        81.0        81.0        81.0
Cabrera High School         84.0        84.0        84.0        84.0
Figueroa High School        81.0        81.0        81.0        81.0
Ford High School            81.0        81.0        80.0        81.0
Griffin High School         83.0        84.0        84.0        84.0
Hernandez High School       81.0        81.0        81.0        81.0
Holden High School          84.0        83.0        84.0        85.0
Huang High School           81.0        82.0        81.0        80.0
Johnson High School         81.0        81.0        81.0        81.0
Pena High School            84.0        84.0        84.0        85.0
Rodriguez High School       81.0        81.0        81.0        80.0
Shelton High School         84.0        83.0        84.0        83.0
Thomas High School          84.0  

In [197]:
# Calculate the average reading score for 12th grade in each school
grade_12th_data = student_data[student_data['grade'] == '12th']

avg_reading_score_12th = grade_12th_data.groupby('school_name')['reading_score'].mean()

print(avg_reading_score_12th)



school_name
Bailey High School       80.912451
Cabrera High School      84.287958
Figueroa High School     81.384863
Ford High School         80.662338
Griffin High School      84.013699
Hernandez High School    80.857143
Holden High School       84.698795
Huang High School        80.305983
Johnson High School      81.227564
Pena High School         84.591160
Rodriguez High School    80.376426
Shelton High School      82.781671
Thomas High School       83.831361
Wilson High School       84.317673
Wright High School       84.073171
Name: reading_score, dtype: float64


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [205]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [227]:
#spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# new column for Spending Range (Per Student)
per_school_summary['Spending Range (Per Student)'] = pd.cut(per_school_summary['Per Student Budget'], bins=spending_bins, labels=group_names)

# Group by the spending range and calculate 
spending_summary = per_school_summary.groupby('Spending Range (Per Student)', observed=True)[
    ['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', 'Overall Passing Rate']
].mean()

# formatting
spending_summary = spending_summary.reset_index()

# Display the summary table
print(spending_summary)

  Spending Range (Per Student)  Average Math Score  Average Reading Score  \
0                        <$585           83.455399              83.933814   
1                     $585-615           83.599686              83.885211   
2                     $615-645           79.079225              81.891436   
3                     $645-675           76.997210              81.027843   

   % Passing Math  % Passing Reading  Overall Passing Rate  
0       93.460096          96.610877             95.035486  
1       94.230858          95.900287             95.065572  
2       75.668212          86.106569             80.887391  
3       66.164813          81.133951             73.649382  


### Scores by School Size

* Perform the same operations as above, based on school size.

In [245]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [252]:

# Create a new column for School Size 
per_school_summary['School Size'] = pd.cut(per_school_summary['Total Students'], bins=size_bins, labels=size_group_names)

# Grouped average math and reading scores
size_math_scores = per_school_summary.groupby('School Size', observed=True)['Average Math Score'].mean()
size_reading_scores = per_school_summary.groupby('School Size', observed=True)['Average Reading Score'].mean()

# Calculate the number of students passing math for each school
math_pass_size = grouped_by_size[grouped_by_size['math_score'] >= 70].groupby('School Size', observed=True).size()

# Calculate the number of students passing reading for each school
read_pass_size = grouped_by_size[grouped_by_size['reading_score'] >= 70].groupby('School Size', observed=True).size()

# Calculate the total number of students in each school size
total_students_size = grouped_by_size.groupby('School Size', observed=True).size()

# will calc math and reading passing per school (hopefully)
size_passing_math = (math_pass_size / total_students_size) * 100
size_passing_reading = (read_pass_size / total_students_size) * 100

#overall passing rates

# My summary DataFrame
size_summary = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": size_overall_passing
}).reset_index()

# will display code
print(size_summary)


          School Size  Average Math Score  Average Reading Score  \
0       Small (<1000)           83.821598              83.929843   
1  Medium (1000-2000)           83.374684              83.864438   
2   Large (2000-5000)           77.746417              81.344493   

   % Passing Math  % Passing Reading  % Overall Passing  
0       93.952484          96.040317          94.996400  
1       93.616522          96.773058          95.194790  
2       68.652380          82.125158          75.388769  


Look for the total count of test scores that pass 70% or higher


In [241]:
# Calculate the overall passing rate for different school size
# Define size bins and group names
size_bins = [0, 1000, 2000, 5000]
size_group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Create a new column for School Size category in per_school_summary
per_school_summary['School Size'] = pd.cut(per_school_summary['Total Students'], bins=size_bins, labels=size_group_names)

# Group the data by school size for the whole dataset
grouped_by_size = school_data_complete.copy()
grouped_by_size['School Size'] = pd.cut(grouped_by_size['size'], bins=size_bins, labels=size_group_names)

# Calculate the number of students passing math for each school size category
math_pass_size = grouped_by_size[grouped_by_size['math_score'] >= 70].groupby('School Size', observed=True).size()

# Calculate the number of students passing reading for each school size category
read_pass_size = grouped_by_size[grouped_by_size['reading_score'] >= 70].groupby('School Size', observed=True).size()

# Calculate the total number of students in each school size category
total_students_size = grouped_by_size.groupby('School Size', observed=True).size()

# Calculate the % Passing Math and % Passing Reading for each school size
perc_passing_math_size = (math_pass_size / total_students_size) * 100
perc_passing_reading_size = (read_pass_size / total_students_size) * 100

# Calculate the overall passing rate (average of math and reading passing rates) for each size category
overall_passing_rate_size = (perc_passing_math_size + perc_passing_reading_size) / 2

#my DatafRame
size_summary = pd.DataFrame({
    'Total Students': total_students_size,
    '% Passing Math': perc_passing_math_size,
    '% Passing Reading': perc_passing_reading_size,
    'Overall Passing Rate': overall_passing_rate_size
}).reset_index()

# Display the summary table
print(size_summary)



          School Size  Total Students  % Passing Math  % Passing Reading  \
0       Small (<1000)            1389       93.952484          96.040317   
1  Medium (1000-2000)            8522       93.616522          96.773058   
2   Large (2000-5000)           29259       68.652380          82.125158   

   Overall Passing Rate  
0             94.996400  
1             95.194790  
2             75.388769  


### Scores by School Type

* Perform the same operations as above, based on school type.

In [262]:
# Create bins and groups, school type {'Charter', 'District'}
school_type_bins = ['Charter', 'District']
group_names = ['Charter', 'District']


Find counts of the passing 70 or higher score for the both test
